In [1]:
import time
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colormaps as cm
from package import saveload

In [2]:
figsize=(12, 4)

In [3]:
def calculate_mean_betty(df: pd.DataFrame, array, dim=None):
    """
    Calculate mean betty numbers
    
    Parameters:
    -----------
    df : DataFrame with columns ['Birth', 'Death', 'Dimension', 'Space']
    
    array : list or np.array
        Array of values
        
    dim : int or None
        Dimension of topological space
        If that's None< that will be maximal dimension from dfs['Dimension']
    
    Returns:
    --------
    res : np.array shape (len(array), dim)
        Array of 
    """
    if dim is None:
        dim = df['Dimension'].max()
    res = np.zeros([len(array), dim])
    for i in range(len(array)):
        val = array[i]
        dfi = df[(df['Birth'] <= val)&(df['Death'] > val)]
        dfi = dfi.groupby(['Space', 'Dimension'], as_index=False).count()[['Space', 'Dimension', 'Birth']]
        dfi = dfi.groupby('Dimension', as_index=False).mean()
        index = dfi['Dimension'].values
        vals = dfi['Birth'].values
        res[i, index] = vals
    return res

In [4]:
def calculate_mean_EC(df: pd.DataFrame, array):
    """
    Calculate mean betty numbers
    
    Parameters:
    -----------
    df : DataFrame with columns ['Birth', 'Death', 'Dimension', 'Space']
    
    array : list or np.array
        Array of values
        
    
    Returns:
    --------
    res : np.array length len(array)
        Array of 
    """
    res = np.zeros(len(array))
    for i in range(len(array)):
        val = array[i]
        dfi = df[(df['Birth'] <= val)&(df['Death'] > val)][['Space', 'Dimension', 'Birth']]
        dfi = dfi.groupby(['Space', 'Dimension'], as_index=False).count()
        dfi.columns = ['Space', 'Dimension', 'Count']
        dfi['Summand EC'] = (-1)**dfi['Dimension'] * dfi['Count']
        dfi = dfi.groupby('Space', as_index=False).sum()['Summand EC']
        res[i] = dfi.mean()
    res[np.isnan(res)] = 0
    return res

# Hexogonal case $PVLM(A_2)$

$A_2 \equiv A_2^*$, but $A_2 \neq A_2^*$

In [ ]:
filename_A2 = 'lab-results/lab_hexagonal_tessellation_on_torus2d_modern_aggregated/size{0}.csv'
sizes_A2 = np.arange(3, 129)

dfs_A2 = [pd.read_csv(filename_A2.format(size)) for size in sizes_A2]
dfs_A2 = [df[df['Death'] == np.inf][['Birth', 'Dimension', 'Space']] for df in dfs_A2]
for i in range(len(sizes_A2)):
    dfs_A2[i]['Size'] = sizes_A2[i]
df_A2 = pd.concat(dfs_A2)
df_A2['Number'] = np.tile(np.arange(3), len(df_A2)//3)
df_A2 = df_A2[df_A2['Number'] > 0]

In [ ]:
df_A2mean = df_A2.groupby(['Size', 'Number'], as_index=False).mean()
df_A2mean.head()

In [ ]:
xticks_A2 = np.arange(3, 129, 12.5) // 1


In [ ]:
births1_A2 = df_A2mean[df_A2mean['Number'] == 1]['Birth']
births2_A2 = df_A2mean[df_A2mean['Number'] == 2]['Birth']

plt.figure(figsize=figsize)
plt.plot(sizes_A2, births1_A2, linewidth=4, label='1st births')
plt.plot(sizes_A2, births2_A2, linewidth=4, label='2nd births')
plt.legend()
plt.xticks(xticks_A2)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=figsize)
plt.plot(sizes_A2, abs(births1_A2.values - births2_A2.values), linewidth=4, label='$|t_1^{perc} - t_2^{perc}|$')
plt.legend()
plt.xticks(xticks_A2)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=figsize)
plt.plot(sizes_A2, births1_A2.values, linewidth=4, label='$t_1^{perc}$')
plt.plot(sizes_A2, 1 - births2_A2.values, linewidth=4, label='$1 - t_2^{perc}$')
plt.legend()
plt.xticks(xticks_A2)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=figsize)
plt.plot(sizes_A2, abs((1 - births1_A2.values) - births1_A2.values), linewidth=4, label='$|(1 - t_1^{perc}) - t_1^{perc}|$')
plt.legend()
plt.xticks(xticks_A2)
plt.grid(True)
plt.savefig('LaTeX/pics/difference_dual_limits_A2.png')

# The case of dula lattice $A_3\equiv\text{fcc}$ and $A_3^*\equiv\text{bcc}$

In [ ]:
sizes_fcc = np.arange(3, 28)
experiments_fcc = 100
filename_fcc = 'lab-results/lab_voronoiD3_tessellation_on_torus3d_aggregated/size{0}.csv'
clock = time.perf_counter()
dfs_fcc = [pd.read_csv(filename_fcc.format(size)) for size in sizes_fcc]
print('Loaded in {0: .4f} seconds.'.format(time.perf_counter() - clock))
for i in range(len(sizes_fcc)):
    dfs_fcc[i]['Size'] = sizes_fcc[i]
df_fcc = [i[i['Death'] == np.inf] for i in dfs_fcc]
df_fcc = pd.concat(df_fcc)
df_fcc['Number'] = np.tile(np.arange(7), len(df_fcc)//7)
df_fcc.head(16)

In [ ]:
sizes_bcc = np.arange(3, 32)
experiments_bcc = 100
filename_bcc = 'lab-results/lab_voronoiBCC_tessellation_on_torus3d_aggregated/size{0}.csv'
clock = time.perf_counter()
dfs_bcc = [pd.read_csv(filename_bcc.format(size)) for size in sizes_bcc]
print('Loaded in {0: .4f} seconds.'.format(time.perf_counter() - clock))
for i in range(len(sizes_bcc)):
    dfs_bcc[i]['Size'] = sizes_bcc[i]
df_bcc = [i[i['Death'] == np.inf] for i in dfs_bcc]
df_bcc = pd.concat(df_bcc)
df_bcc['Number'] = np.tile(np.arange(7), len(df_bcc)//7)
df_bcc.head(16)

In [ ]:
df_fccmean = df_fcc.groupby(['Size', 'Number'], as_index=False).mean()
df_fccmean.head(8)

In [ ]:
df_bccmean = df_bcc.groupby(['Size', 'Number'], as_index=False).mean()
df_bccmean.head(8)

In [ ]:
if len(sizes_fcc) < len(sizes_bcc):
    sizes_fccbcc = sizes_fcc
else:
    sizes_fccbcc = sizes_bcc

In [ ]:
t1_fcc = df_fccmean[df_fccmean['Number'] == 1]['Birth'].values[:len(sizes_fccbcc)]
t2_fcc = df_fccmean[df_fccmean['Number'] == 4]['Birth'].values[:len(sizes_fccbcc)]

In [ ]:
t1_bcc = df_bccmean[df_bccmean['Number'] == 1]['Birth'].values[:len(sizes_fccbcc)]
t2_bcc = df_bccmean[df_bccmean['Number'] == 4]['Birth'].values[:len(sizes_fccbcc)]

In [ ]:
plt.figure(figsize=figsize)

plt.plot(sizes_fccbcc, abs((1 - t1_fcc) - t2_bcc), linewidth=4, label='$\\left|\\left(1 - t_1^{perc}(fcc)\\right) - t_2^{perc}(bcc)\\right|$')
plt.plot(sizes_fccbcc, abs((1 - t1_bcc) - t2_fcc), linewidth=4, label='$\\left|\\left(1 - t_1^{perc}(bcc)\\right) - t_2^{perc}(fcc)\\right|$')

plt.legend()
plt.xticks(np.arange(3, 28, 3))
plt.grid(True)
plt.savefig('LaTeX/pics/difference_dual_limits_fcc_bcc.png')